In [156]:
import time
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import datetime
import geopy.distance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


from scipy import stats

sns.set(color_codes=True)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load Extra TFT Data

In [160]:
tft_items = None
tft_traits = None
tft_champs = None
with open('./tft_extra_data/items.json') as json_file:
    tft_items = json.load(json_file)

with open('./tft_extra_data/traits.json') as json_file:
    tft_traits = json.load(json_file)

with open('./tft_extra_data/champions.json') as json_file:
    tft_champs = json.load(json_file)


# API Calls

In [128]:
URL = 'https://na1.api.riotgames.com/tft/league/v1/challenger'
api_key = 'RGAPI-a9c6b14d-2f84-4a3e-a7b8-df8f8b62da32'
headers = {'X-Riot-Token': api_key}


### Initial Challenger Data Pull

In [129]:
# Initial Challenger Data Pull
response = requests.get(URL, headers=headers)

In [14]:
summoner_names = []
league_points = []
for entry in response.json()['entries']:
    summoner_names.append(entry['summonerName'])
    league_points.append(entry['leaguePoints'])

In [34]:
player_df = pd.DataFrame(data = {'summoner_name': summoner_names, 'league_points': league_points})

In [35]:
player_df.sort_values(by=['league_points'], ascending=False, inplace=True)
player_df.reset_index(inplace=True)
player_df.drop(['index'], axis=1, inplace=True)

In [57]:
i = 0
player_uuids = []

In [190]:
player_df.head()

,summoner_name,league_points,puuid
0,GrandVice8,1334,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...
1,DeliciousMilkGG,1207,PY8453ZFDl78q0BN6ZsYU8m4U7E-4Sr2y6YcT5lNeo0th-...
2,Silliest,1113,zdUfZ0_rhBEI-Gx6ai33CxZyCZymPuL0HvZEBSTAw762Lh...
3,Agon,1096,Mtiy5xQq2B08E8UKrWt_AcHemRq40W5xJQKfC6wIONGMCJ...
4,Krmx,1087,VJQMuXGE5493waSe6PFxLrn2zPrQfMAi7ZDTBMwU7rXt3r...


### Search users by Summoner Name

In [70]:
URL_name = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/'

for index in range(i+1, len(player_df['summoner_name'])):
    name = player_df.iloc[index]['summoner_name']
    print(index, name)
    
    response = requests.get(URL_name + name, headers=headers)
    
    print(response.json()['puuid'])
    
    player_uuids.append(response.json()['puuid'])
    i = index



199 GreenbeingTFT
MmQ9S18WFQcAwNH1zAFuoiJeTKj9S0KYe6dl6Fe2YVuQkMvsNX3uHKYCM97gqlic-rHCiZE5czr4sQ


In [72]:
player_df['puuid'] = player_uuids

### Get Last 10 Matches for each player

In [119]:
match_ids_df = pd.DataFrame(columns=['puuid', 'match_id'])
URL_match_ids = 'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/'
num_api_calls = 0
i = 0

In [120]:
for index in range(i, len(player_df['puuid'])):
    player_id = player_df.iloc[index]['puuid']
    if num_api_calls >= 100:
        time.sleep(120)
        num_api_calls = 0
    response = requests.get(URL_match_ids + player_id + '/ids?count=10', headers=headers)
    num_api_calls += 1

    for match in response.json():
        match_ids_df = match_ids_df.append({'puuid': player_id, 'match_id': match}, ignore_index=True)

    i = index
    

In [191]:
match_ids_df.head()

,puuid,match_id
0,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249805878
1,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249791129
2,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249755964
3,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249723025
4,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249669510


### Get Game Data from Each Match

In [165]:
# Got match IDS
# Look up matches and obtain relevant information 
# Only those with top 4 placings?
game_info_df = pd.DataFrame(columns=['puuid', 
                                     'match_id', 
                                     'game_length', 
                                     'gold_left', 
                                     'last_round', 
                                     'level', 
                                     'placement',
                                     'players_eliminated',
                                     'time_eliminated',
                                     'total_damage_to_players',
                                     'traits', 
                                     'units'])
URL_game = 'https://americas.api.riotgames.com/tft/match/v1/matches/'
num_api_calls = 0

In [189]:
i = 1780

In [187]:
for index in range(i, len(match_ids_df['match_id'])):
    match_id = match_ids_df.iloc[index]['match_id']
    if num_api_calls >= 100:
        time.sleep(120)
        num_api_calls = 0
    response = requests.get(URL_game + match_id, headers=headers)
    num_api_calls += 1
    data = response.json()['info']
    game_len = data['game_length']
    
    for p in data['participants']:
        traits = []
        units = []
        for t in p['traits']:
            traits.append((t['name'], t['tier_current']))
        
        for u in p['units']:
            units.append((u['name'], u['rarity'], u['tier'], u['items']))
        
        game_info_df = game_info_df.append({'puuid': p['puuid'],
                             'match_id': match_id,
                             'game_length': game_len,
                             'gold_left': p['gold_left'],
                             'last_round': p['last_round'],
                             'level': p['level'],
                             'placement': p['placement'],
                             'players_eliminated': p['players_eliminated'],
                             'time_eliminated': p['time_eliminated'],
                             'total_damage_to_players': p['total_damage_to_players'],
                             'traits': traits,
                             'units': units
                        }, ignore_index=True)
    print(len(game_info_df.index))
    i = index

In [192]:
game_info_df

,puuid,match_id,game_length,gold_left,last_round,level,placement,players_eliminated,time_eliminated,total_damage_to_players,traits,units
0,HbYsPpRS3X2RaQ8n5Sm9n2gKU80f927PzBk_krw4kdyRG6...,NA1_3249805878,1959.947266,5,31,8,4,0,1714.102783,95,"[(Berserker, 2), (Desert, 0), (Electric, 1), (...","[(Renekton, 0, 2, []), (DrMundo, 2, 2, []), (S..."
1,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249805878,1959.947266,0,35,9,2,3,1951.685059,146,"[(Avatar, 1), (Electric, 3), (Inferno, 1), (Li...","[(Annie, 3, 2, []), (Zed, 4, 2, [24, 66, 16]),..."
2,HGeDGOQrklxGiScizUCLq_BVn5gTYBy-DldOuy38XcaKap...,NA1_3249805878,1959.947266,0,31,7,5,0,1711.923218,72,"[(Berserker, 0), (Desert, 2), (Inferno, 0), (M...","[(Nocturne, 2, 2, [19, 19, 23]), (Sivir, 2, 2,..."
3,VyCpwiJ3MifHEREwjj0cSAXHQIjFGd7UtWMBG9ZvCdlGso...,NA1_3249805878,1959.947266,30,26,7,7,0,1445.723633,55,"[(Inferno, 1), (Light, 0), (Mountain, 0), (Mys...","[(Kindred, 2, 2, [44, 22, 39]), (Malzahar, 1, ..."
4,UsiShYgxH3X5_pNeJjE-EpBhbR4vevorLcs83aI8ZSQFaZ...,NA1_3249805878,1959.947266,4,33,8,3,0,1819.271362,129,"[(Avatar, 1), (Desert, 1), (Inferno, 0), (Meta...","[(Yasuo, 1, 2, []), (Sivir, 2, 2, [25, 46, 46]..."
5,BXPswgAxjGfAfQHvaF-m7CtUdbBvZQhcviLqZwZDyDZbDn...,NA1_3249805878,1959.947266,50,28,8,6,0,1567.998779,80,"[(Inferno, 1), (Light, 0), (Mage, 0), (Mystic,...","[(Nami, 4, 1, [25, 57]), (Thresh, 1, 2, [67]),..."
6,zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMl...,NA1_3249805878,1959.947266,8,24,7,8,0,1324.369019,31,"[(Desert, 1), (Inferno, 0), (Light, 0), (Metal...","[(Aatrox, 2, 2, []), (Nocturne, 2, 2, [19, 28,..."
7,BXZMHIfauv8M3BuaqO6wZnxQXrN2flWKj1GhJ6nNQeP3DD...,NA1_3249805878,1959.947266,1,35,9,1,4,1951.685059,176,"[(Avatar, 1), (Crystal, 0), (Electric, 0), (In...","[(Malphite, 3, 2, []), (Lux, 5, 1, []), (Brand..."
8,ELtY718rHfroXSGxxjTLh35qHtSQhlp2f4K50eq2xOtHbl...,NA1_3249791129,2157.252197,2,35,8,2,2,2148.779785,122,"[(Desert, 1), (Metal, 1), (Mountain, 0), (Myst...","[(Qiyana, 2, 2, [34]), (Sivir, 2, 2, [46, 46])..."
9,OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1d...,NA1_3249791129,2157.252197,4,35,9,1,2,2148.779785,195,"[(Avatar, 1), (Electric, 0), (Inferno, 0), (Li...","[(Lux, 5, 2, [44, 44, 37]), (Malphite, 3, 2, [..."


In [193]:
game_info_df.to_csv('./complete_match_data.csv')
match_ids_df.to_csv('./match_ids.csv')
player_df.to_csv('./player_data.csv')